### Import required libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


#### Get branch data from FDIC website. This is available @ https://www5.fdic.gov/sod/ShowFileWithStats1.asp?strFileName=ALL_2018.zip
Save this in an excel

In [3]:
# This is a big file, so I have copied this into my local folder from URL mentioned above
data = pd.read_excel('C:/peri personal/TRAINING/data science/9. applied data science capstone/project data/ALL_branch_data_2018.xlsx')  
data.head()

,YEAR,CERT,BRNUM,UNINUMBR,NAMEFULL,ADDRESBR,CITYBR,CNTYNAMB,STALPBR,ZIPBR,BRCENM,CONSOLD,BRSERTYP,DEPSUMBR,BKMO,CBSA_DIV_NAMB,CITY2BR,CNTRYNAB,CNTYNUMB,CSABR,CSANAMBR,DIVISIONB,MSABR,MSANAMB,METROBR,MICROBR,NAMEBR,NECTABR,NECNAMB,PLACENUM,SIMS_ACQUIRED_DATE,SIMS_ESTABLISHED_DATE,SIMS_LATITUDE,SIMS_LONGITUDE,SIMS_DESCRIPTION,SIMS_PROJECTION,STCNTYBR,STNAMEBR,STNUMBR,HCTMULT,RSSDHCR,NAMEHCR,CITYHCR,STALPHCR,RSSDID,UNIT,ADDRESS,CITY,STALP,ZIP,ASSET,BKCLASS,CALL,CHARTER,CHRTAGNN,CHRTAGNT,CLCODE,CNTRYNA,DENOVO,DEPDOM,DEPSUM,DOCKET,ESCROW,FDICDBS,FDICNAME,FED,FEDNAME,INSAGNT1,INSURED,INSBRDD,INSBRTS,OCCDIST,OCCNAME,REGAGNT,SPECGRP,SPECDESC,STCNTY,STNAME,USA
0,2018,14,0,6,State Street Bank and Trust Company,1 Lincoln St. Fl 1,Boston,Suffolk,MA,2111,M,NaN,11,112389655,1,"Boston, MA",Boston,United States,25,148,"Boston-Worcester-Providence, MA-RI-NH-CT",14454,14460,"Boston-Cambridge-Newton, MA-NH",1,0,State Street Bank And Trust Company,0,NaN,0,NaT,01/01/1792,42.3526,-71.0579,M,100,25025,Massachusetts,25,MULT,1111435,STATE STREET CORPORATION,BOSTON,MA,35301,1.0,1 Lincoln St. Fl 1,Boston,MA,2111,245244124,SM,CALL,STATE,State Agency,STATE,13,United States,0.0,112389655,112389655,14278,NaN,2,New York,1,Boston,DIF,CB,NaN,NaN,1,Northeast District,FED,1,INTERNATIONAL,25025,Massachusetts,1
1,2018,35,0,17,AuburnBank,100 North Gay Street,Auburn,Lee,AL,36830,M,NaN,11,435618,1,NaN,Auburn,United States,81,194,"Columbus-Auburn-Opelika, GA-AL",0,12220,"Auburn-Opelika, AL",1,0,Auburnbank,0,NaN,0,NaT,1907-01-03 00:00:00,32.6071,-85.4798,POINTADDRESS,100,1081,Alabama,1,ONE,1129533,"AUBURN NATIONAL BANCORPORATION, INC.",AUBURN,AL,749635,0.0,100 North Gay Street,Auburn,AL,36830,812640,SM,CALL,STATE,State Agency,STATE,13,United States,0.0,722956,722956,9177,NaN,5,Atlanta,6,Atlanta,DIF,CB,NaN,NaN,5,Southwest District,FED,4,COMMERCIAL LENDING,1081,Alabama,1
2,2018,35,13,531906,AuburnBank,132 Fob James Drive,Valley,Chambers,AL,36854,NaN,NaN,11,34307,0,NaN,Valley,United States,17,194,"Columbus-Auburn-Opelika, GA-AL",0,0,NaN,0,1,Valley Branch,0,NaN,0,NaT,2011-12-12 00:00:00,32.8154,-85.1789,M,100,1017,Alabama,1,ONE,1129533,"AUBURN NATIONAL BANCORPORATION, INC.",AUBURN,AL,749635,0.0,100 North Gay Street,Auburn,AL,36830,812640,SM,CALL,STATE,State Agency,STATE,13,United States,0.0,722956,0,9177,NaN,5,Atlanta,6,Atlanta,DIF,CB,NaN,NaN,5,Southwest District,FED,4,COMMERCIAL LENDING,1081,Alabama,1
3,2018,35,7,361391,AuburnBank,1351 S. Donahue Drive,Auburn,Lee,AL,36830,NaN,NaN,11,15246,0,NaN,Auburn,United States,81,194,"Columbus-Auburn-Opelika, GA-AL",0,12220,"Auburn-Opelika, AL",1,0,South Donahue Branch,0,NaN,0,NaT,2000-09-20 00:00:00,32.6314,-85.4980,STREETADDRESS,92.43,1081,Alabama,1,ONE,1129533,"AUBURN NATIONAL BANCORPORATION, INC.",AUBURN,AL,749635,0.0,100 North Gay Street,Auburn,AL,36830,812640,SM,CALL,STATE,State Agency,STATE,13,United States,0.0,722956,0,9177,NaN,5,Atlanta,6,Atlanta,DIF,CB,NaN,NaN,5,Southwest District,FED,4,COMMERCIAL LENDING,1081,Alabama,1
4,2018,35,10,478897,AuburnBank,1851 Frederick Road,Opelika,Lee,AL,36801,NaN,NaN,11,17564,0,NaN,Opelika,United States,81,194,"Columbus-Auburn-Opelika, GA-AL",0,12220,"Auburn-Opelika, AL",1,0,Frederick Road,0,NaN,0,NaT,2007-07-12 00:00:00,32.6227,-85.4028,STREETADDRESS,100,1081,Alabama,1,ONE,1129533,"AUBURN NATIONAL BANCORPORATION, INC.",AUBURN,AL,749635,0.0,100 North Gay Street,Auburn,AL,36830,812640,SM,CALL,STATE,State Agency,STATE,13,United States,0.0,722956,0,9177,NaN,5,Atlanta,6,Atlanta,DIF,CB,NaN,NaN,5,Southwest District,FED,4,COMMERCIAL LENDING,1081,Alabama,1


#### Filter rows to get data for California and store this in another dataframe

In [4]:
df = data.loc[data['STALPBR'] == "CA"]
df.reset_index(inplace = True,drop=True)
df.head()

,YEAR,CERT,BRNUM,UNINUMBR,NAMEFULL,ADDRESBR,CITYBR,CNTYNAMB,STALPBR,ZIPBR,BRCENM,CONSOLD,BRSERTYP,DEPSUMBR,BKMO,CBSA_DIV_NAMB,CITY2BR,CNTRYNAB,CNTYNUMB,CSABR,CSANAMBR,DIVISIONB,MSABR,MSANAMB,METROBR,MICROBR,NAMEBR,NECTABR,NECNAMB,PLACENUM,SIMS_ACQUIRED_DATE,SIMS_ESTABLISHED_DATE,SIMS_LATITUDE,SIMS_LONGITUDE,SIMS_DESCRIPTION,SIMS_PROJECTION,STCNTYBR,STNAMEBR,STNUMBR,HCTMULT,RSSDHCR,NAMEHCR,CITYHCR,STALPHCR,RSSDID,UNIT,ADDRESS,CITY,STALP,ZIP,ASSET,BKCLASS,CALL,CHARTER,CHRTAGNN,CHRTAGNT,CLCODE,CNTRYNA,DENOVO,DEPDOM,DEPSUM,DOCKET,ESCROW,FDICDBS,FDICNAME,FED,FEDNAME,INSAGNT1,INSURED,INSBRDD,INSBRTS,OCCDIST,OCCNAME,REGAGNT,SPECGRP,SPECDESC,STCNTY,STNAME,USA
0,2018,622,30,446207,Amalgamated Bank,"255 California Street, Suite 600",San Francisco,San Francisco,CA,94111,NaN,NaN,11,340822,0,San Francisco-Redwood City-South San Francisco...,San Francisco,United States,75,488,"San Jose-San Francisco-Oakland, CA",41884,41860,"San Francisco-Oakland-Hayward, CA",1,0,New Resource Bank Main Office Branch,0,NaN,0,2018-05-18,2006-09-19 00:00:00,37.7932,-122.3995,PointAddress,100,6075,California,6,ONE,4400336,WORKERS UNITED,PHILADELPHIA,PA,661308,0.0,275 Seventh Avenue,New York,NY,10001,4617720,NM,CALL,STATE,State Agency,STATE,21,United States,0.0,3971251,0,9297,NaN,2,New York,2,New York,DIF,CB,NaN,NaN,1,Northeast District,FDIC,4,COMMERCIAL LENDING,36061,New York,1
1,2018,628,7370,522285,"JPMorgan Chase Bank, National Association","1 Grand Avenue, Bldg 65, Room 103",San Luis Obispo,San Luis Obispo,CA,93407,NaN,NaN,11,14526,0,NaN,San Luis Obispo,United States,79,0,NaN,0,42020,"San Luis Obispo-Paso Robles-Arroyo Grande, CA",1,0,Cal Poly Student Union Bk Ctr Branch,0,NaN,0,NaT,2011-09-08 00:00:00,35.2960,-120.6530,POSTAL,100,6079,California,6,MULT,1039502,JPMORGAN CHASE & CO.,NEW YORK,NY,852218,0.0,1111 Polaris Parkway,Columbus,OH,43240,2167700000,N,CALL,FED,Comptroller of the Currency,OCC,3,United States,0.0,1274091000,0,0,NaN,9,Chicago,4,Cleveland,DIF,CB,NaN,NaN,3,Central District,OCC,1,INTERNATIONAL,39041,Ohio,1
2,2018,628,4512,275790,"JPMorgan Chase Bank, National Association",10 South El Camino Real,Millbrae,San Mateo,CA,94030,NaN,NaN,11,361307,0,San Francisco-Redwood City-South San Francisco...,Millbrae,United States,81,488,"San Jose-San Francisco-Oakland, CA",41884,41860,"San Francisco-Oakland-Hayward, CA",1,0,Millbrae Branch,0,NaN,0,2008-09-25,1919-01-01 00:00:00,37.5984,-122.3866,PointAddress,100,6081,California,6,MULT,1039502,JPMORGAN CHASE & CO.,NEW YORK,NY,852218,0.0,1111 Polaris Parkway,Columbus,OH,43240,2167700000,N,CALL,FED,Comptroller of the Currency,OCC,3,United States,0.0,1274091000,0,0,NaN,9,Chicago,4,Cleveland,DIF,CB,NaN,NaN,3,Central District,OCC,1,INTERNATIONAL,39041,Ohio,1
3,2018,628,4762,273856,"JPMorgan Chase Bank, National Association",10 Town Center Pkwy,Santee,San Diego,CA,92071,NaN,NaN,11,125999,0,NaN,Santee,United States,73,0,NaN,0,41740,"San Diego-Carlsbad, CA",1,0,Santee Branch,0,NaN,0,2008-09-25,1934-11-01 00:00:00,32.8391,-116.9883,M,US_RoofTop,6073,California,6,MULT,1039502,JPMORGAN CHASE & CO.,NEW YORK,NY,852218,0.0,1111 Polaris Parkway,Columbus,OH,43240,2167700000,N,CALL,FED,Comptroller of the Currency,OCC,3,United States,0.0,1274091000,0,0,NaN,9,Chicago,4,Cleveland,DIF,CB,NaN,NaN,3,Central District,OCC,1,INTERNATIONAL,39041,Ohio,1
4,2018,628,7296,521723,"JPMorgan Chase Bank, National Association","100 N La Cienega Blvd, Ste B114",Los Angeles,Los Angeles,CA,90048,NaN,NaN,11,55545,0,"Los Angeles-Long Beach-Glendale, CA",Los Angeles,United States,37,348,"Los Angeles-Long Beach, CA",31084,31080,"Los Angeles-Long Beach-Anaheim, CA",1,0,Beverly And La Cienega Branch,0,NaN,0,NaT,2011-05-04 00:00:00,34.0741,-118.3764,M,100,6037,California,6,MULT,1039502,JPMORGAN CHASE & CO.,NEW YORK,NY,852218,0.0,1111 Polaris Parkway,Columbus,OH,43240,2167700000,N,CALL,FED,Comptroller of the Currency,OCC,3,United States,0.0,1274091000,0,0,NaN,9,Chicago,4,Cleveland,DIF,CB,NaN,NaN,3,Central District,OCC,1,INTERNATIONAL,39041,Ohio,1


#### Sort the data frame in ascending order of branches with maximum deposits

In [5]:
df = df.sort_values(by=['DEPSUMBR'],ascending=False)
df.reset_index(inplace = True,drop=True)
df.head()

,YEAR,CERT,BRNUM,UNINUMBR,NAMEFULL,ADDRESBR,CITYBR,CNTYNAMB,STALPBR,ZIPBR,BRCENM,CONSOLD,BRSERTYP,DEPSUMBR,BKMO,CBSA_DIV_NAMB,CITY2BR,CNTRYNAB,CNTYNUMB,CSABR,CSANAMBR,DIVISIONB,MSABR,MSANAMB,METROBR,MICROBR,NAMEBR,NECTABR,NECNAMB,PLACENUM,SIMS_ACQUIRED_DATE,SIMS_ESTABLISHED_DATE,SIMS_LATITUDE,SIMS_LONGITUDE,SIMS_DESCRIPTION,SIMS_PROJECTION,STCNTYBR,STNAMEBR,STNUMBR,HCTMULT,RSSDHCR,NAMEHCR,CITYHCR,STALPHCR,RSSDID,UNIT,ADDRESS,CITY,STALP,ZIP,ASSET,BKCLASS,CALL,CHARTER,CHRTAGNN,CHRTAGNT,CLCODE,CNTRYNA,DENOVO,DEPDOM,DEPSUM,DOCKET,ESCROW,FDICDBS,FDICNAME,FED,FEDNAME,INSAGNT1,INSURED,INSBRDD,INSBRTS,OCCDIST,OCCNAME,REGAGNT,SPECGRP,SPECDESC,STCNTY,STNAME,USA
0,2018,3510,1015,198460,"Bank of America, National Association",315 Montgomery St,San Francisco,San Francisco,CA,94104,NaN,NaN,11,80804486,0,San Francisco-Redwood City-South San Francisco...,San Francisco,United States,75,488,"San Jose-San Francisco-Oakland, CA",41884,41860,"San Francisco-Oakland-Hayward, CA",1,0,San Francisco Main Branch,0,NaN,0,NaT,1969-08-25 00:00:00,37.7920,-122.4026,POINTADDRESS,100,6075,California,6,MULT,1073757,BANK OF AMERICA CORPORATION,CHARLOTTE,NC,480228,0.0,100 North Tryon St,Charlotte,NC,28202,1759530000,N,CALL,FED,Comptroller of the Currency,OCC,3,United States,0.0,1312701000,0,0,NaN,5,Atlanta,5,Richmond,DIF,CB,NaN,NaN,1,Northeast District,OCC,9,ALL OTHER > $1 BILLION,37119,North Carolina,1
1,2018,24735,0,17167,Silicon Valley Bank,3003 Tasman Drive,Santa Clara,Santa Clara,CA,95054,M,NaN,11,41157121,1,NaN,Santa Clara,United States,85,488,"San Jose-San Francisco-Oakland, CA",0,41940,"San Jose-Sunnyvale-Santa Clara, CA",1,0,Silicon Valley Bank,0,NaN,0,NaT,1983-10-17 00:00:00,37.4034,-121.9803,M,US_Streets,6085,California,6,ONE,1031449,SVB FINANCIAL GROUP,SANTA CLARA,CA,802866,0.0,3003 Tasman Drive,Santa Clara,CA,95054,55073093,SM,CALL,STATE,State Agency,STATE,13,United States,0.0,45910483,45910483,0,NaN,14,San Francisco,12,San Francisco,DIF,CB,NaN,NaN,4,Midwest District,FED,9,ALL OTHER > $1 BILLION,6085,California,1
2,2018,22826,211,842,"MUFG Union Bank, National Association",445 South Figueroa Street,Los Angeles,Los Angeles,CA,90071,NaN,NaN,11,28232734,0,"Los Angeles-Long Beach-Glendale, CA",Los Angeles,United States,37,348,"Los Angeles-Long Beach, CA",31084,31080,"Los Angeles-Long Beach-Anaheim, CA",1,0,Los Angeles Main Branch,0,NaN,0,1996-04-01,1914-07-01 00:00:00,34.0527,-118.2567,M,US_RoofTop,6037,California,6,ONE,2961897,"MITSUBISHI UFJ FINANCIAL GROUP, INC.",TOKYO,NaN,212465,0.0,400 California Street,San Francisco,CA,94104,123787395,N,CALL,FED,Comptroller of the Currency,OCC,3,Japan,0.0,86497122,0,0,NaN,14,San Francisco,12,San Francisco,DIF,CB,NaN,NaN,4,Midwest District,OCC,4,COMMERCIAL LENDING,6075,California,1
3,2018,59017,0,497251,First Republic Bank,111 Pine Street,San Francisco,San Francisco,CA,94111,M,NaN,11,19143243,1,San Francisco-Redwood City-South San Francisco...,San Francisco,United States,75,488,"San Jose-San Francisco-Oakland, CA",41884,41860,"San Francisco-Oakland-Hayward, CA",1,0,First Republic Bank,0,NaN,0,NaT,2010-07-01 00:00:00,37.7923,-122.3989,POINTADDRESS,100,6075,California,6,NONE,0,NaN,NaN,NaN,4114567,0.0,111 Pine Street,San Francisco,CA,94111,93851460,NM,CALL,STATE,State Agency,STATE,21,NaN,0.0,72771950,72771950,0,NaN,14,San Francisco,12,San Francisco,DIF,CB,NaN,NaN,4,Midwest District,FDIC,4,COMMERCIAL LENDING,6075,California,1
4,2018,3511,603,188692,"Wells Fargo Bank, National Association",1 California Street,San Francisco,San Francisco,CA,94111,NaN,NaN,11,19008316,0,San Francisco-Redwood City-South San Francisco...,San Francisco,United States,75,488,"San Jose-San Francisco-Oakland, CA",41884,41860,"San Francisco-Oakland-Hayward, CA",1,0,Market-California Branch,0,NaN,0,1986-05-30,1922-11-28 00:00:00,37.7935,-122.3970,M,US_RoofTop,6075,California,6,MULT,1120754,WELLS FARGO & COMPANY,SAN FRANCISCO,CA,451965,0.0,101 N. Phillips Avenue,Sioux Falls,SD,57104,1675077000,N,CALL,FED,Comptroller of the Currency,OCC,3,United Stat

#### Get zip codes of the top 30 branches

In [6]:
df1=df.head(30)
df1.head()

,YEAR,CERT,BRNUM,UNINUMBR,NAMEFULL,ADDRESBR,CITYBR,CNTYNAMB,STALPBR,ZIPBR,BRCENM,CONSOLD,BRSERTYP,DEPSUMBR,BKMO,CBSA_DIV_NAMB,CITY2BR,CNTRYNAB,CNTYNUMB,CSABR,CSANAMBR,DIVISIONB,MSABR,MSANAMB,METROBR,MICROBR,NAMEBR,NECTABR,NECNAMB,PLACENUM,SIMS_ACQUIRED_DATE,SIMS_ESTABLISHED_DATE,SIMS_LATITUDE,SIMS_LONGITUDE,SIMS_DESCRIPTION,SIMS_PROJECTION,STCNTYBR,STNAMEBR,STNUMBR,HCTMULT,RSSDHCR,NAMEHCR,CITYHCR,STALPHCR,RSSDID,UNIT,ADDRESS,CITY,STALP,ZIP,ASSET,BKCLASS,CALL,CHARTER,CHRTAGNN,CHRTAGNT,CLCODE,CNTRYNA,DENOVO,DEPDOM,DEPSUM,DOCKET,ESCROW,FDICDBS,FDICNAME,FED,FEDNAME,INSAGNT1,INSURED,INSBRDD,INSBRTS,OCCDIST,OCCNAME,REGAGNT,SPECGRP,SPECDESC,STCNTY,STNAME,USA
0,2018,3510,1015,198460,"Bank of America, National Association",315 Montgomery St,San Francisco,San Francisco,CA,94104,NaN,NaN,11,80804486,0,San Francisco-Redwood City-South San Francisco...,San Francisco,United States,75,488,"San Jose-San Francisco-Oakland, CA",41884,41860,"San Francisco-Oakland-Hayward, CA",1,0,San Francisco Main Branch,0,NaN,0,NaT,1969-08-25 00:00:00,37.7920,-122.4026,POINTADDRESS,100,6075,California,6,MULT,1073757,BANK OF AMERICA CORPORATION,CHARLOTTE,NC,480228,0.0,100 North Tryon St,Charlotte,NC,28202,1759530000,N,CALL,FED,Comptroller of the Currency,OCC,3,United States,0.0,1312701000,0,0,NaN,5,Atlanta,5,Richmond,DIF,CB,NaN,NaN,1,Northeast District,OCC,9,ALL OTHER > $1 BILLION,37119,North Carolina,1
1,2018,24735,0,17167,Silicon Valley Bank,3003 Tasman Drive,Santa Clara,Santa Clara,CA,95054,M,NaN,11,41157121,1,NaN,Santa Clara,United States,85,488,"San Jose-San Francisco-Oakland, CA",0,41940,"San Jose-Sunnyvale-Santa Clara, CA",1,0,Silicon Valley Bank,0,NaN,0,NaT,1983-10-17 00:00:00,37.4034,-121.9803,M,US_Streets,6085,California,6,ONE,1031449,SVB FINANCIAL GROUP,SANTA CLARA,CA,802866,0.0,3003 Tasman Drive,Santa Clara,CA,95054,55073093,SM,CALL,STATE,State Agency,STATE,13,United States,0.0,45910483,45910483,0,NaN,14,San Francisco,12,San Francisco,DIF,CB,NaN,NaN,4,Midwest District,FED,9,ALL OTHER > $1 BILLION,6085,California,1
2,2018,22826,211,842,"MUFG Union Bank, National Association",445 South Figueroa Street,Los Angeles,Los Angeles,CA,90071,NaN,NaN,11,28232734,0,"Los Angeles-Long Beach-Glendale, CA",Los Angeles,United States,37,348,"Los Angeles-Long Beach, CA",31084,31080,"Los Angeles-Long Beach-Anaheim, CA",1,0,Los Angeles Main Branch,0,NaN,0,1996-04-01,1914-07-01 00:00:00,34.0527,-118.2567,M,US_RoofTop,6037,California,6,ONE,2961897,"MITSUBISHI UFJ FINANCIAL GROUP, INC.",TOKYO,NaN,212465,0.0,400 California Street,San Francisco,CA,94104,123787395,N,CALL,FED,Comptroller of the Currency,OCC,3,Japan,0.0,86497122,0,0,NaN,14,San Francisco,12,San Francisco,DIF,CB,NaN,NaN,4,Midwest District,OCC,4,COMMERCIAL LENDING,6075,California,1
3,2018,59017,0,497251,First Republic Bank,111 Pine Street,San Francisco,San Francisco,CA,94111,M,NaN,11,19143243,1,San Francisco-Redwood City-South San Francisco...,San Francisco,United States,75,488,"San Jose-San Francisco-Oakland, CA",41884,41860,"San Francisco-Oakland-Hayward, CA",1,0,First Republic Bank,0,NaN,0,NaT,2010-07-01 00:00:00,37.7923,-122.3989,POINTADDRESS,100,6075,California,6,NONE,0,NaN,NaN,NaN,4114567,0.0,111 Pine Street,San Francisco,CA,94111,93851460,NM,CALL,STATE,State Agency,STATE,21,NaN,0.0,72771950,72771950,0,NaN,14,San Francisco,12,San Francisco,DIF,CB,NaN,NaN,4,Midwest District,FDIC,4,COMMERCIAL LENDING,6075,California,1
4,2018,3511,603,188692,"Wells Fargo Bank, National Association",1 California Street,San Francisco,San Francisco,CA,94111,NaN,NaN,11,19008316,0,San Francisco-Redwood City-South San Francisco...,San Francisco,United States,75,488,"San Jose-San Francisco-Oakland, CA",41884,41860,"San Francisco-Oakland-Hayward, CA",1,0,Market-California Branch,0,NaN,0,1986-05-30,1922-11-28 00:00:00,37.7935,-122.3970,M,US_RoofTop,6075,California,6,MULT,1120754,WELLS FARGO & COMPANY,SAN FRANCISCO,CA,451965,0.0,101 N. Phillips Avenue,Sioux Falls,SD,57104,1675077000,N,CALL,FED,Comptroller of the Currency,OCC,3,United Stat

#### Plot these branches on california map. For this get the coordinates of california first

In [7]:
address = 'San Jose'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of California are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of California are 37.3361905, -121.8905833.


Keep relavent columns into another dataframe

In [8]:
zip_data=[]
column_names=['CERT','BRNUM','NAMEFULL','ADDRESS','CITYBR','CNTYNAMB','STALPBR','ZIPBR','DEPSUMBR','SIMS_LATITUDE','SIMS_LONGITUDE']
zip_data=pd.DataFrame(columns=column_names)

In [9]:
#zip_data = [d.reset_index(drop=True) for d in (zip_data)]
#zip_data.index=df2.index
zip_data=df1[['CERT','BRNUM','NAMEFULL','ADDRESS','CITYBR','CNTYNAMB','STALPBR','ZIPBR','DEPSUMBR','SIMS_LATITUDE','SIMS_LONGITUDE']]
zip_data.head()

,CERT,BRNUM,NAMEFULL,ADDRESS,CITYBR,CNTYNAMB,STALPBR,ZIPBR,DEPSUMBR,SIMS_LATITUDE,SIMS_LONGITUDE
0,3510,1015,"Bank of America, National Association",100 North Tryon St,San Francisco,San Francisco,CA,94104,80804486,37.7920,-122.4026
1,24735,0,Silicon Valley Bank,3003 Tasman Drive,Santa Clara,Santa Clara,CA,95054,41157121,37.4034,-121.9803
2,22826,211,"MUFG Union Bank, National Association",400 California Street,Los Angeles,Los Angeles,CA,90071,28232734,34.0527,-118.2567
3,59017,0,First Republic Bank,111 Pine Street,San Francisco,San Francisco,CA,94111,19143243,37.7923,-122.3989
4,3511,603,"Wells Fargo Bank, National Association",101 N. Phillips Avenue,San Francisco,San Francisco,CA,94111,19008316,37.7935,-122.3970


#### Create a map of California with top branches superimposed on top

In [10]:
# create map of california using latitude and longitude values
map_california = folium.Map(location=[latitude, longitude], zoom_start=6)

# add markers to map
for lat, lng, city, cert in zip(zip_data['SIMS_LATITUDE'], zip_data['SIMS_LONGITUDE'], zip_data['CITYBR'], zip_data['CERT']):
    label = '{}, {}'.format(cert, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_california)  
    
map_california

#### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

### Define four square credentials and version

In [11]:
CLIENT_ID = 'M12EJXOUAKCCU3AF5JOSVSHZF1YCWHNK44VEWNMB4EQASPWG' # your Foursquare ID
CLIENT_SECRET = 'C2XQO0NS2SWZF5LMJLMQSQM34IHE0XVKTZ1EPKCMARHP23J4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: M12EJXOUAKCCU3AF5JOSVSHZF1YCWHNK44VEWNMB4EQASPWG
CLIENT_SECRET:C2XQO0NS2SWZF5LMJLMQSQM34IHE0XVKTZ1EPKCMARHP23J4


#### Let's explore the first neighborhood in our dataframe.

Get the city name

In [12]:
zip_data.loc[0,'CITYBR']

'San Francisco'

Get the city's latitude and longitude values.

In [13]:
city_latitude = zip_data.loc[0, 'SIMS_LATITUDE'] # neighborhood latitude value
city_longitude = zip_data.loc[0, 'SIMS_LONGITUDE'] # neighborhood longitude value

city_name = zip_data.loc[0, 'CITYBR'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               city_latitude, 
                                                               city_longitude))

Latitude and longitude values of San Francisco are 37.792, -122.4026.


#### Now, let's get the top 100 venues that are in San Francisco within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.

In [14]:
# type your answer here
#search_query = 'Marble Hill'
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    city_latitude, 
    city_longitude, 
    VERSION, 
    radius, 
    LIMIT)
url




'https://api.foursquare.com/v2/venues/explore?client_id=M12EJXOUAKCCU3AF5JOSVSHZF1YCWHNK44VEWNMB4EQASPWG&client_secret=C2XQO0NS2SWZF5LMJLMQSQM34IHE0XVKTZ1EPKCMARHP23J4&ll=37.792,-122.4026&v=20180605&radius=500&limit=100'

Send the GET request and examine the resutls

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c909cb2db04f50db03cb028'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Financial District',
  'headerFullLocation': 'Financial District, San Francisco',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 203,
  'suggestedBounds': {'ne': {'lat': 37.796500004500004,
    'lng': -122.39691615817343},
   'sw': {'lat': 37.7874999955, 'lng': -122.40828384182659}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '58cc4d2fe0adac17bbf4838e',
       'name': 'Pushkin',
       'location': {'address': '380 Bush St',
        'crossStreet': 'at Kearny St',
        'lat': 37.79094301071348,
        'lng':

All the information is in the items key. Before we proceed, let's write functionto get_category_type

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now clean the json and structure it into a pandas dataframe.

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Pushkin,Russian Restaurant,37.790943,-122.403877
1,Blue Bottle Coffee,Coffee Shop,37.791320,-122.400983
2,Blue Bottle Coffee,Coffee Shop,37.792548,-122.404515
3,Equinox Pine Street,Gym,37.792036,-122.401289
4,Sababa,Mediterranean Restaurant,37.791185,-122.404219


Check how many venues were returned by Foursquare?

In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


# 2. Explore Neighborhoods in the cities having maximum deposits in the banks

create a function to repeat the same process to all the cities in these top 30 cities

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now write the code to run the above function on each city with top bank deposits  and create a new dataframe called top_venues.

In [21]:
top_venues = getNearbyVenues(names=zip_data['CITYBR'],
                                   latitudes=zip_data['SIMS_LATITUDE'],
                                   longitudes=zip_data['SIMS_LONGITUDE']
                                  )



San Francisco
Santa Clara
Los Angeles
San Francisco
San Francisco
Santa Monica
San Francisco
Beverly Hills
San Ramon
Los Angeles
Sacramento
El Monte
San Francisco
Los Angeles
Los Angeles
Palo Alto
Los Angeles
San Diego
Pasadena
San Diego
San Francisco
San Jose
Walnut Creek
San Francisco
Los Angeles
San Francisco
Menlo Park
San Diego
Los Angeles
San Francisco


Let's check the size of the resulting dataframe

In [22]:
print(top_venues.shape)
top_venues.head()

(2126, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,San Francisco,37.792,-122.4026,Pushkin,37.790943,-122.403877,Russian Restaurant
1,San Francisco,37.792,-122.4026,Blue Bottle Coffee,37.791320,-122.400983,Coffee Shop
2,San Francisco,37.792,-122.4026,Blue Bottle Coffee,37.792548,-122.404515,Coffee Shop
3,San Francisco,37.792,-122.4026,Equinox Pine Street,37.792036,-122.401289,Gym
4,San Francisco,37.792,-122.4026,Sababa,37.791185,-122.404219,Mediterranean Restaurant


Let's check how many venues were returned for each top city

In [23]:
top_venues.reset_index(inplace = True,drop=True)

In [24]:
top_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Beverly Hills,100,100,100,100,100,100
El Monte,14,14,14,14,14,14
Los Angeles,536,536,536,536,536,536
Menlo Park,6,6,6,6,6,6
Palo Alto,78,78,78,78,78,78
Pasadena,87,87,87,87,87,87
Sacramento,28,28,28,28,28,28
San Diego,228,228,228,228,228,228
San Francisco,835,835,835,835,835,835


Let's find out how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(top_venues['Venue Category'].unique())))

There are 227 uniques categories.


## 3. Analyze Each City

In [26]:
# one hot encoding
top_onehot = pd.get_dummies(top_venues[['Venue Category']], prefix="", prefix_sep="")

# add City column back to dataframe
top_onehot['City'] = top_venues['City'] 
top_onehot.head()

,ATM,Acai House,Accessories Store,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Store,Big Box Store,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doctor's Office,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fondue Restaurant,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Heliport,Hobby Shop,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wagashi Place,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,City
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,San Francisco
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [27]:
top_onehot.columns[-1]

'City'

In [28]:
# move City column to the first column
fixed_columns = [top_onehot.columns[-1]] + list(top_onehot.columns[:-1])
top_onehot = top_onehot[fixed_columns]

top_onehot.head()

,City,ATM,Acai House,Accessories Store,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Store,Big Box Store,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doctor's Office,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fondue Restaurant,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Heliport,Hobby Shop,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wagashi Place,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,San Francisco,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,San Francisco,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Let's examine the new dataframe size.

In [29]:
top_onehot.shape

(2126, 228)

#### Next, let's group rows by city and by taking the mean of the frequency of occurrence of each category

In [30]:
top_grouped = top_onehot.groupby('City').mean().reset_index()
top_grouped

,City,ATM,Acai House,Accessories Store,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Store,Big Box Store,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doctor's Office,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fondue Restaurant,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Heliport,Hobby Shop,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Leather Goods Store,Library,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Ramen Restaurant,Rental Car Location,Resort,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Snack Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wagashi Place,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Beverly Hills,0.000000,0.000000,0.010000,0.00,0.00,0.00,0.00,0.070000,0.000000,0.000000,0.010000,0.000000,0.010000,0.010000,0.0000,0.000000,0.000000,0.010000,0.00,0.020000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.010000,0.010000,0.000000,0.000000,0.00000,0.080000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.020000,0.000000,0.020000,0.030000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.01,0.000000,0.000000,0.000

Let's confirm the new size

In [31]:
top_grouped.shape

(14, 228)

#### Let's print each city along with the top 5 most common venues

In [32]:
num_top_venues = 5

for hood in top_grouped['City']:
    print("----"+hood+"----")
    temp = top_grouped[top_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Beverly Hills----
                 venue  freq
0             Boutique  0.10
1       Clothing Store  0.08
2  American Restaurant  0.07
3   Italian Restaurant  0.05
4        Women's Store  0.05


----El Monte----
                  venue  freq
0  Fast Food Restaurant  0.21
1  Gym / Fitness Center  0.14
2      Cuban Restaurant  0.07
3           Coffee Shop  0.07
4                  Café  0.07


----Los Angeles----
                venue  freq
0         Coffee Shop  0.06
1      Sandwich Place  0.05
2               Hotel  0.05
3  Italian Restaurant  0.04
4                Café  0.04


----Menlo Park----
               venue  freq
0        Coffee Shop  0.33
1  Convention Center  0.17
2              Hotel  0.17
3         Playground  0.17
4               Café  0.17


----Palo Alto----
                venue  freq
0      Ice Cream Shop  0.05
1         Salad Place  0.04
2               Hotel  0.04
3   French Restaurant  0.03
4  Chinese Restaurant  0.03


----Pasadena----
                 venue  f

#### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each City.

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = top_grouped['City']

for ind in np.arange(top_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(top_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beverly Hills,Boutique,Clothing Store,American Restaurant,Women's Store,Italian Restaurant,Café,Hotel,Juice Bar,Coffee Shop,Jewelry Store
1,El Monte,Fast Food Restaurant,Gym / Fitness Center,Ice Cream Shop,Coffee Shop,Pizza Place,Cuban Restaurant,Sandwich Place,Shipping Store,Cajun / Creole Restaurant,Café
2,Los Angeles,Coffee Shop,Hotel,Sandwich Place,Italian Restaurant,Café,Mexican Restaurant,New American Restaurant,American Restaurant,Sushi Restaurant,French Restaurant
3,Menlo Park,Coffee Shop,Convention Center,Hotel,Playground,Café,Dumpling Restaurant,Football Stadium,Food Truck,Food Court,Fondue Restaurant
4,Palo Alto,Ice Cream Shop,Hotel,Salad Place,Yoga Studio,Japanese Restaurant,Coffee Shop,Tea Room,Mexican Restaurant,Electronics Store,Pizza Place


## 4.Cluster cities

Run k-means to cluster the cities into 5 clusters.

In [35]:
# set number of clusters
kclusters = 5

top_grouped_clustering = top_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(top_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 4, 1, 1, 1, 1, 1, 1])

In [36]:
city_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beverly Hills,Boutique,Clothing Store,American Restaurant,Women's Store,Italian Restaurant,Café,Hotel,Juice Bar,Coffee Shop,Jewelry Store
1,El Monte,Fast Food Restaurant,Gym / Fitness Center,Ice Cream Shop,Coffee Shop,Pizza Place,Cuban Restaurant,Sandwich Place,Shipping Store,Cajun / Creole Restaurant,Café
2,Los Angeles,Coffee Shop,Hotel,Sandwich Place,Italian Restaurant,Café,Mexican Restaurant,New American Restaurant,American Restaurant,Sushi Restaurant,French Restaurant
3,Menlo Park,Coffee Shop,Convention Center,Hotel,Playground,Café,Dumpling Restaurant,Football Stadium,Food Truck,Food Court,Fondue Restaurant
4,Palo Alto,Ice Cream Shop,Hotel,Salad Place,Yoga Studio,Japanese Restaurant,Coffee Shop,Tea Room,Mexican Restaurant,Electronics Store,Pizza Place


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each city.

In [37]:
# add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

top_merged = zip_data

# merge top cities with to add latitude/longitude for each neighborhood
top_merged = top_merged.join(city_venues_sorted.set_index('City'), on='CITYBR')

top_merged.head() # check the last columns!

,CERT,BRNUM,NAMEFULL,ADDRESS,CITYBR,CNTYNAMB,STALPBR,ZIPBR,DEPSUMBR,SIMS_LATITUDE,SIMS_LONGITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3510,1015,"Bank of America, National Association",100 North Tryon St,San Francisco,San Francisco,CA,94104,80804486,37.7920,-122.4026,1,Coffee Shop,Sandwich Place,Food Truck,Gym,Café,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Men's Store,Cocktail Bar
1,24735,0,Silicon Valley Bank,3003 Tasman Drive,Santa Clara,Santa Clara,CA,95054,41157121,37.4034,-121.9803,3,Intersection,Italian Restaurant,Hotel,Coffee Shop,Sushi Restaurant,Electronics Store,Football Stadium,Auditorium,Café,Gym Pool
2,22826,211,"MUFG Union Bank, National Association",400 California Street,Los Angeles,Los Angeles,CA,90071,28232734,34.0527,-118.2567,1,Coffee Shop,Hotel,Sandwich Place,Italian Restaurant,Café,Mexican Restaurant,New American Restaurant,American Restaurant,Sushi Restaurant,French Restaurant
3,59017,0,First Republic Bank,111 Pine Street,San Francisco,San Francisco,CA,94111,19143243,37.7923,-122.3989,1,Coffee Shop,Sandwich Place,Food Truck,Gym,Café,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Men's Store,Cocktail Bar
4,3511,603,"Wells Fargo Bank, National Association",101 N. Phillips Avenue,San Francisco,San Francisco,CA,94111,19008316,37.7935,-122.3970,1,Coffee Shop,Sandwich Place,Food Truck,Gym,Café,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Men's Store,Cocktail Bar


Finally, let's visualize the resulting clusters

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=6)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(top_merged['SIMS_LATITUDE'], top_merged['SIMS_LONGITUDE'], top_merged['CITYBR'], top_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, lets examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, let's then assign a name to each cluster.

#### Cluster 1

In [39]:
top_merged.loc[top_merged['Cluster Labels'] == 0, top_merged.columns[[1] + list(range(5, top_merged.shape[1]))]]

,BRNUM,CNTYNAMB,STALPBR,ZIPBR,DEPSUMBR,SIMS_LATITUDE,SIMS_LONGITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,291,Los Angeles,CA,91731,7955662,34.0712,-118.0662,0,Fast Food Restaurant,Gym / Fitness Center,Ice Cream Shop,Coffee Shop,Pizza Place,Cuban Restaurant,Sandwich Place,Shipping Store,Cajun / Creole Restaurant,Café


#### Cluster 2

In [40]:
top_merged.loc[top_merged['Cluster Labels'] == 1, top_merged.columns[[1] + list(range(5, top_merged.shape[1]))]]

,BRNUM,CNTYNAMB,STALPBR,ZIPBR,DEPSUMBR,SIMS_LATITUDE,SIMS_LONGITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1015,San Francisco,CA,94104,80804486,37.7920,-122.4026,1,Coffee Shop,Sandwich Place,Food Truck,Gym,Café,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Men's Store,Cocktail Bar
2,211,Los Angeles,CA,90071,28232734,34.0527,-118.2567,1,Coffee Shop,Hotel,Sandwich Place,Italian Restaurant,Café,Mexican Restaurant,New American Restaurant,American Restaurant,Sushi Restaurant,French Restaurant
3,0,San Francisco,CA,94111,19143243,37.7923,-122.3989,1,Coffee Shop,Sandwich Place,Food Truck,Gym,Café,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Men's Store,Cocktail Bar
4,603,San Francisco,CA,94111,19008316,37.7935,-122.3970,1,Coffee Shop,Sandwich Place,Food Truck,Gym,Café,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Men's Store,Cocktail Bar
5,88,Los Angeles,CA,90404,14523672,34.0307,-118.4741,1,Coffee Shop,Food Truck,American Restaurant,Gym,Sandwich Place,Print Shop,ATM,Park,Shipping Store,Butcher
6,0,San Francisco,CA,94104,12924000,37.7927,-122.4037,1,Coffee Shop,Sandwich Place,Food Truck,Gym,Café,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Men's Store,Cocktail Bar
7,165,Los Angeles,CA,90210,11618226,34.0671,-118.4060,1,Boutique,Clothing Store,American Restaurant,Women's Store,Italian Restaurant,Café,Hotel,Juice Bar,Coffee Shop,Jewelry Store
9,1070,Los Angeles,CA,90017,10384073,34.0493,-118.2598,1,Coffee Shop,Hotel,Sandwich Place,Italian Restaurant,Café,Mexican Restaurant,New American Restaurant,American Restaurant,Sushi Restaurant,French Restaurant
10,2050,Sacramento,CA,95814,8020930,38.5784,-121.4998,1,Coffee Shop,Gym / Fitness Center,Steakhouse,Italian Restaurant,American Restaurant,Café,Thai Restaurant,Basketball Stadium,Bar,Chinese Restaurant
12,5907,San Francisco,CA,94104,7893081,37.7933,-122.4024,1,Coffee Shop,Sandwich Place,Food Truck,Gym,Café,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Men's Store,Cocktail Bar


#### Cluster 3

In [41]:
top_merged.loc[top_merged['Cluster Labels'] == 2, top_merged.columns[[1] + list(range(5, top_merged.shape[1]))]]

,BRNUM,CNTYNAMB,STALPBR,ZIPBR,DEPSUMBR,SIMS_LATITUDE,SIMS_LONGITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,857,Contra Costa,CA,94583,11544883,37.7717,-121.963,2,Furniture / Home Store,Business Service,Yoga Studio,Dumpling Restaurant,Football Stadium,Food Truck,Food Court,Fondue Restaurant,Filipino Restaurant,Fast Food Restaurant


#### Cluster 4

In [42]:
top_merged.loc[top_merged['Cluster Labels'] == 3, top_merged.columns[[1] + list(range(5, top_merged.shape[1]))]]

,BRNUM,CNTYNAMB,STALPBR,ZIPBR,DEPSUMBR,SIMS_LATITUDE,SIMS_LONGITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,0,Santa Clara,CA,95054,41157121,37.4034,-121.9803,3,Intersection,Italian Restaurant,Hotel,Coffee Shop,Sushi Restaurant,Electronics Store,Football Stadium,Auditorium,Café,Gym Pool


#### Cluster 5

In [43]:
top_merged.loc[top_merged['Cluster Labels'] == 4, top_merged.columns[[1] + list(range(5, top_merged.shape[1]))]]

,BRNUM,CNTYNAMB,STALPBR,ZIPBR,DEPSUMBR,SIMS_LATITUDE,SIMS_LONGITUDE,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,62,San Mateo,CA,94025,4103093,37.4212,-122.2057,4,Coffee Shop,Convention Center,Hotel,Playground,Café,Dumpling Restaurant,Football Stadium,Food Truck,Food Court,Fondue Restaurant
